# AI: How easy is it to create a helpdesk assistant using Transformers library

*Imagine being the developer on your squad, crafting a bot that doesn’t just spit out automatic responses but actually gets what your users are asking. That’s the kind of magic we’re talking about here. And guess what? It’s not as complicated as it sounds, especially with this guide.*

---

First off, if you’re just starting with AI or have poked around a bit, you’ve probably heard of the OpenAI API and the Transformers library. Both are like having your own Gandalf for building AI stuff, but they cater to different kinds of wizards. OpenAI API is like the express train to AI-ville. It’s super user-friendly, meaning you don’t need to know the ins and outs of AI to make something cool. It’s perfect for getting your app to chat away or understand user commands without getting your hands dirty with the nitty-gritty of model training.

## Here’s the game plan:
Preparing the state: Start by installing the right dependencies to use the Transform library. Ensure you’ve got Python installed — most spells in the AI world are cast with it. If you haven’t, head over to the Python website and follow the instructions for your operating system.

*** On the colab platform set Your Collab Runtime type to T4 GPU in the Menu: Runtime > Change runtime type > Hardware accelerator ***

Next, open your command line or terminal and cast the following spell to summon the Transformers library into your realm (or just install it):

In [29]:
!pip install transformers

**Choose Your Wand** — CPU or GPU?: Here’s where you pick the power source for your magic. If you’re just tinkering or learning, running the models on your CPU might be fine. But, if you’re planning to do some heavy spellwork or want super-fast responses, you’ll want to harness the power of a GPU. In this way consider to additional dependencies:

In [30]:
!pip install accelerate -U

*On the colab platform you can skip this step*

**NVIDIA CUDA for GPU Wizards:** If you’ve chosen the GPU path and have an NVIDIA graphics card, you’ll need to install CUDA and cuDNN to get everything running smoothly. These are like the arcane texts that let your GPU understand how to conjure AI spells. Head over to the NVIDIA website, find the versions compatible with your graphics card, and follow the installation guides.

[NVidia CUDA Toolkit](https://developer.nvidia.com/cuda-toolkit)

## Pick Your Potion: Choosing the right model.
There are tons out there, but for a helpdesk assistant, you’ll want something good with text, like BERT or GPT-2. We’re going with GPT-2 in this tutorial.

In [31]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, GPT2Config

## Training Montage
Next up, you’ll need to train your model. This sounds epic because it is. You’re essentially teaching your bot to understand and respond to queries by feeding it examples. We need to consider having a starting dataset to teach the AI and witness the magic unfold. Below, there’s a small set of hypothetical Q&A to use in our tutorial, just copy them and save in a text file in your project folder, for example with the name dataset.txt

```
# My smartphone is not turning on. What should I do?
# I'm sorry to hear that. Let's try a few troubleshooting steps. First, make sure the battery is charged. If it doesn't work, press and hold the power button for at least 10 seconds to force a restart.

# How can I transfer photos from my phone to my computer?
# To transfer photos, connect your phone to the computer using a USB cable. Once connected, open the file explorer on your computer, find your phone, and navigate to the "DCIM" folder. Copy the photos from there to your computer.

# I forgot my phone's PIN. How can I unlock it?
# No worries! If you've set up a Google or Apple account on your phone, you can use the "Forgot PIN" or "Forgot Password" option on the lock screen. Follow the on-screen instructions to reset your PIN.

# How do I update the software on my smartphone?
# To update your phone's software, go to Settings > System > Software Update. If there's an update available, follow the on-screen prompts to download and install it. Ensure your phone is connected to Wi-Fi during the update.

# My phone's camera is blurry. What can I do to fix it?
# Blurriness in the camera might be due to smudges on the lens. Gently clean the lens with a microfiber cloth. If the issue persists, check if there are any software updates for the camera app in your phone's app store.

# How can I set up email on my smartphone?
# To set up email, go to Settings > Accounts > Add Account. Select "Email" and enter your email address and password. The phone will automatically configure the email settings. If you're having trouble, double-check your email provider's settings.

# My phone is running slow. Any tips to speed it up?
# Slow performance can be due to multiple apps running in the background. Close unused apps and clear cache from Settings > Storage > Cached data. If the issue persists, consider uninstalling unnecessary apps or freeing up storage space.

# How can I disable notifications for a specific app?
# To disable notifications for a specific app, go to Settings > Apps > [App Name]. Look for the "Notifications" section and toggle off the switch. This will stop notifications from that app.

# My phone's screen is cracked. What should I do?
# I'm sorry to hear that. If the screen is still functional, you may consider using a screen protector temporarily. For a permanent solution, contact the phone manufacturer or visit an authorized service center for screen repair or replacement.

# How do I factory reset my smartphone?
# Performing a factory reset will erase all data on your phone. To do this, go to Settings > System > Reset > Factory data reset. Follow the on-screen instructions and ensure you have backed up important data before proceeding.
```





## Get your mind ready
We’re going through a recipe for some digital wizardry, specifically for training an AI to be your next-gen helpdesk assistant.

Kick-off with the Tokenizer: Imagine you’ve got a blender (that’s your tokenizer), and you’re throwing in some text to chop up into bits that your AI can easily digest. We’re using the GPT2TokenizerFast here, tuned to ‘gpt2’, which means it’s ready to handle text the way GPT-2 likes it.

In [32]:
# Initialize a tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')

**Whip Up the Dataset:** Now, think of the dataset as your main ingredient. This line tells your setup, “Hey, grab the helpdesk conversations from this text file.” It’s like you’re prepping your veggies before you start cooking. The block_size=128 bit is about how big a bite you want your AI to chew on in one go. Here, consider changing the file_path attribute to point to your previously created dataset.

In [33]:
# Prepare the dataset
# Using Google Drive as a storage for your dataset
# from google.colab import drive
# drive.mount("/content/drive")
# dataset = TextDataset(tokenizer=tokenizer, file_path="./drive/My Drive/Colab Notebooks/dataset.txt", block_size=128)
dataset = TextDataset(tokenizer=tokenizer, file_path="./dataset.txt", block_size=128)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


**Data Collator — The Mixer:** This part’s all about getting your data ready for the oven. The DataCollatorForLanguageModeling is like saying, “Mix up my text chunks so they’re just right for training.” Setting mlm=False tells it we’re not hiding any of the text from our model; it gets to see everything full on.

In [34]:
# Prepare the data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

**Model Time — Preheating the Oven:** Here’s where you pick your oven model. We’re rolling with GPT-2, a pretty fancy choice if you’re into making your AI understand and generate human-like text. We skip the part where we build the oven from scratch (GPT2Config) and go straight to using a preheated one with GPT2LMHeadModel.from_pretrained(‘gpt2’).

In [35]:
# Initialize a blank model with GPT-2 architecture
# config = GPT2Config.from_pretrained('gpt2')
# model = GPT2LMHeadModel(config)
# Load the pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

**Training Arguments — Setting the Timer:** This chunk is like dialing in the settings on your oven. You’re telling it how long to cook (num_train_epochs=100, which is like saying, “Let’s run this 100 times to make sure it’s really got it”), how hot (or how many examples to look at once with per_device_train_batch_size=32), and when to check if the dish is done (save_steps=10_000). Oh, and save_total_limit=2 is kinda like saying, “I’ve only got space for two leftovers in the fridge, so toss anything older.”

In [36]:
# Specify the training arguments
training_args = TrainingArguments(
    output_dir="./gpt2_pretrained",  # The output directory
    overwrite_output_dir=True,  # Overwrite the content of the output directory
    num_train_epochs=100,  # Number of training epochs
    per_device_train_batch_size=32,  # Batch size for training
    save_steps=10_000,  # Number of updates steps before two checkpoint saves
    save_total_limit=2,  # Limit the total amount of checkpoints and delete the older checkpoints
)

**Start the Training:** With everything in place, you kick off the training. This is when your AI starts learning from the data, making sense of the patterns, understanding questions, and figuring out how to generate answers. Imagine it’s like baking a cake; you’ve set the temperature and time, now you let it transform inside the oven. The training process is where the magic happens, turning your raw data into a sophisticated AI model capable of handling real-world queries. It’s a mix of art and science, requiring patience and adjustments in the dataset or on the training arguments to get it just right.

*On the colab platform don't forget to set your runtime type to T4 GPU*

In [37]:
# It's important to have an account on the website https://wandb.ai/site.
# This command will prompt you to input the API Key provided by wandb.ai, either on the welcome screen or in your profile.
!pip install wandb
!wandb login

wandb: Currently logged in as: alanarantes (alanarantes-alan-arantes). Use `wandb login --relogin` to force relogin


In [38]:
import os
os.environ["WANDB_PROJECT"]="custom_helpdesk_assistant"

In [39]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Train the model
trainer.train()

# Save the model
trainer.save_model()


Step,Training Loss


Putting all this together, you’re setting up a state-of-the-art kitchen to whip up an AI that’s gonna handle your helpdesk like a pro. It’s all about prepping your ingredients, mixing them right, and cooking them under the perfect conditions. Bon appétit, tech-style!

## Using your brain
We’re gonna walk through whipping up a batch of smart, on-the-fly responses using some serious coding chops and AI wizardry. Let’s get into the nitty-gritty of this tech recipe.

Gathering the Ingredients: We need make some imports before we start to use our brain previously stored.

In [40]:
import sys
from time import time
import torch

from transformers import GPT2LMHeadModel, GPT2TokenizerFast

from functools import lru_cache

**Important notes:** We’re calling in the big guns from the Transformers library: the GPT-2 LM Head Model and GPT-2 Tokenizer Fast. Think of them as your high-end food processor and slicer, ready to handle anything you throw at them. To enhance performance and avoid redundant calls that yield the same result, we’ve also decided to implement a small cache layer using the @lru_cache annotation.

Continuing with our activities, we initialize our tokenizer, which is like setting up your cutting board and knives to chop up text into manageable pieces our AI can understand.

In [41]:
# Initialize a tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')

We load our pre-trained model, kind of like getting that secret recipe book out. We’re setting up to use either the GPU or CPU, depending on what kind of firepower we have available. This is the equivalent of deciding if you’re cooking on a gas stove or an electric one.

In [42]:
# Load the trained model for inference
model = GPT2LMHeadModel.from_pretrained("./gpt2_pretrained")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device: {device}")
model.to(device)

device: cuda


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

**Getting Down to Business:** We briefly consider making our operations more efficient with model.half() to use FP16 precision, kind of like choosing to prep your ingredients beforehand. But we decide to leave that for another day and keep things straightforward.

Now, here’s where we start cooking for real. We define a generate_response function, which is our recipe for turning questions into answers. This function is like a magic cauldron that takes in a question, stirs it through the AI model, and poof! Out comes a response.

We’re not just blindly cooking here; we’re timing each step of the process — from slicing and dicing the question to simmering it in the AI model and serving up the response. It’s all about efficiency in our kitchen.

When we generate the response, we’re setting some rules like how long we want our answer to be (max_length=50) and making sure it's original (no_repeat_ngram_size=2). It's like seasoning to taste and making sure not to overcook.

In [43]:
# Use FP16 precision
# model = model.half()

# Define a function for generating responses
@lru_cache(maxsize=None)
def generate_response(input_text):
  start_tokenizer = time()
  inputs = tokenizer.encode(input_text, return_tensors='pt')
  inputs = inputs.to(device)
  stop_tokenizer = time()

  start_generate = time()
  outputs = model.generate(inputs, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2, temperature=0.7)
  stop_generate = time()

  start_decode = time()
  full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
  stop_decode = time()

  start_find_response = time()
  guard = full_response.find('User:') if full_response.find('User:') > 0 else len(full_response)
  guard -= len('User:')
  response = full_response[0:guard].strip()
  stop_find_response = time()

  print(f"tokenizer elapsed time {stop_tokenizer - start_tokenizer}")
  print(f"generate elapsed time {stop_generate - start_generate}")
  print(f"decode elapsed time {stop_decode - start_decode}")
  print(f"find_response elapsed time {stop_find_response - start_find_response}")
  return response

**Serving the Dish:** After our AI assistant does its magic, we present the magic — I mean, the answer. We even clean it up a bit, trimming off any excess to make sure it’s just what the user asked for. Finally, we take our test question about a blurry phone camera.

In [44]:
# Use this function in your command-line interface to generate responses to user input
input = "My phone's camera is blurry. What can I do to fix it?"
print(f"input {input}")
print(generate_response(input))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


input My phone's camera is blurry. What can I do to fix it?
tokenizer elapsed time 0.0008115768432617188
generate elapsed time 0.8022818565368652
decode elapsed time 0.00030303001403808594
find_response elapsed time 6.4373016357421875e-06
My phone's camera is blurry. What can I do to fix it?
# Blurriness in the camera might be due to smudges on the lens. Gently clean the lenses with a microfiber cloth. If the
